## Purpose:
Check 2D space charge in Synergia vs envelope equations for free beam expansion.

The Bassetti-Erskine 2D space charge model is used. This is a "frozen" space charge algorithm, which assumes the transverse particle distribution is Gaussian in position and momentum. Synergia also provides self-consistent PIC algorithms. Longitudinal space charge is ignored here. Synergia provides 3D algorithms for both PIC and Bassetti-Erskine.

The initial transverse particle distribution is assumed to be Gaussian. The initial longitudinal particle distribution is assumed to be uniform in position (z), with zero energy spread. The initial particles are read from a local file.

### Number of particles in the beam

1. We work with the peak current, I.
2. The beam consists of N protons, with total charge $Q = N e$.
2. Assume beam has total length L, velocity $\beta$c, and the particles are uniformly distributed in z.
3. The current is then: $I = \frac{Q \beta c}{L}$, and the corresponding number is $N = \frac{I L}{\beta c e}$
4. The number of particles per unit length is $\frac{N}{L} = \frac{I}{\beta c e}$ 

This implies a proton number scaling of $2.85769 \times 10^8 \frac{p}{m \cdot mA}$, which at (for example) 14 mA equates to $4.0 \times 10^{9}$ protons per meter.

### Initial import statements

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys, os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy
import tables
from mpi4py import MPI

### Import physics libraries

In [ ]:
try:
    import rssynergia
except ImportError:
    !pip -q install git+git://github.com/radiasoft/rssynergia

from rssynergia.base_diagnostics import read_bunch
from rssynergia.base_diagnostics import workflow
from rssynergia.base_diagnostics import lfplot
from rssynergia.base_diagnostics import latticework
from rssynergia.base_diagnostics import basic_calcs
from rssynergia.base_diagnostics import pltbunch
from rssynergia.base_diagnostics import elliptic_sp
from rssynergia.base_diagnostics import singleparticle
from rssynergia.base_diagnostics import options
from rssynergia.base_diagnostics import diagplot
from rssynergia.base_diagnostics import utils

from rssynergia.elliptic import elliptic_beam6d
from rssynergia.standard import standard_beam6d

import synergia
import synergia_workflow

### Test problem - Expanding beam in a drift

Drift length:
- 3 m

Beam Parameters:
- 14 mA, 2.5 MeV proton beam
- Emittance: 0.3 mm-mrad normalized
- 64x64x32 mesh (2D Open)
- 25 turns at 0.1 m drift length

### Create workdir and specify Synergia simulation options (default values)

In [ ]:
# Create and populate a Synergia options object
# File I/O
opts = synergia_workflow.Options("zc_drift")
opts.add("output_dir","sc_drift", "Directory for output files", str)
opts.relpath = opts.output_dir
workflow.make_path(opts.output_dir)
opts.add("verbosity", 1, "Verbosity of propagation", int)
opts.add("bunch_file","myBunch.txt","txt file for bunch particles", str)

# Define reference particle to be a proton at 2.5 MeV
total_energy = synergia.foundation.pconstants.proton_mass + 2.5e-3  # [GeV]
four_momentum = synergia.foundation.Four_momentum(synergia.foundation.pconstants.proton_mass, total_energy)
reference_particle = synergia.foundation.Reference_particle(synergia.foundation.pconstants.proton_charge,four_momentum)
opts.gamma = reference_particle.get_gamma()
opts.beta = reference_particle.get_beta()

# beam (physical)
opts.add("emit",9.74e-6, "H0 value corresponding to real sigma horizontal emittance of 0.3 mm-mrad", float)
opts.add("dpop", 0.0, "Delta-p/p spread", float)
opts.add("real_particles", 1.0e11, "Number of real particles", float)
opts.emit_n = 0.3*1.e-6    # 0.3 mm-mrad normalized emittance
opts.emits = [basic_calcs.calc_geometric_emittance(opts.emit_n,opts.beta,opts.gamma)]
dpop = 0.0

# beam (numerical)
opts.add("macro_particles", 50000, "Number of macro particles", int)    
opts.add("spacecharge", True, "whether space charge is on", bool)
opts.add("solver", "2dbassetti-erskine", "other solvers are available", str)

# Lattice
opts.add("steps_per_element",5,"Number of steps per element", int)
opts.add("turns",30,"Number of turns", int)
opts.add("checkpointperiod", 15, "Interval for creating checkpoints", int)
opts.add("radius", 0.5, "aperture radius [m]", float)
opts.add("stepper", "splitoperator", "Simulation stepper, either 'independent','elements','splitoperator','soelements'", str)

### Construct the lattice (a simple drift)

In [ ]:
# specify the drift element
#    note:  above, it is specified that 5 steps are taken per element
#           hence, the integration step size is my_drift_length / 5.
#           above, the number of "turns" is specified as 30
#           hence, the total drift length is 30.*my_drift_length
#           hence, the total number of steps in Synergia is 30*5 = 150
my_drift_length = 0.1  # [m]
drift_element = synergia.lattice.Lattice_element("drift", "drift_element")
drift_element.set_double_attribute("l", my_drift_length)

# instantiate the lattice
lattice = synergia.lattice.Lattice("test", synergia.lattice.Mad8_adaptor_map())
lattice.append(drift_element)
lattice.set_reference_particle(reference_particle)
opts.lattice = lattice

# specify the operator
coll_operator = synergia.collective.Space_charge_2d_bassetti_erskine()

# instantiate the lattice stepper and simulator
map_order = 1
stepper = synergia.simulation.Split_operator_stepper_elements(lattice, map_order,coll_operator, opts.steps_per_element)
opts.lattice_simulator = stepper.get_lattice_simulator()

In [ ]:
#Cleanup any files from previous runs
try:
    os.remove('myBunch.txt')
except OSError:
    pass

files = os.listdir(opts.output_dir)
for file in files:
    if file.endswith('.h5'):
        os.remove(os.path.join(opts.output_dir,file))

files = os.listdir('.')
for file in files:
    if file.endswith('.h5'):
        os.remove(os.path.join('.',file))


## Construct the bunch

In [ ]:
# Bunch options
current = 14.e-3 #mA of current 
rp_perlength = current/(opts.beta*scipy.constants.c*scipy.constants.e)
bunch_length = 2e-2 #effective bunch length 2 mm
real_particles = rp_perlength*bunch_length

opts.emit_n = 6e-7 #We want 0.3 mm-mrad normalized emittance
opts.emits = [basic_calcs.calc_geometric_emittance(opts.emit_n,opts.beta,opts.gamma)] #give this geometric emittance
opts.real_particles = rp_perlength*bunch_length
opts.betae = 0.5 #statically fix beta
opts.alphae = 0.0

# load pre-generated Gaussian bunch from file
bunch = np.loadtxt("myGaussianBunch.txt")
    
bunch[:,4] = bunch_length*(np.random.random(len(bunch)) -0.5) #center at 0
bunch[:,5] = opts.dpop*np.random.randn(1,len(bunch)) #set dp/p

np.savetxt('myBunch.txt',bunch)         #write the bunch to a text file

In [ ]:
#Check emittance
emit = np.sqrt(np.average(bunch[:,0]**2) * np.average(bunch[:,1]**2) - np.average(bunch[:,0]*bunch[:,1])**2)
print "geometric emittance: %s \nnormalized emittance: %s" % (emit, emit * (opts.beta * opts.gamma))

In [ ]:
#read in the bunch
particles_file = opts.bunch_file
bucket_length = bunch_length #set equal
comm = synergia.utils.Commxx(True) #define a communicator
myBunch = read_bunch.read_bunch(particles_file, reference_particle, opts.real_particles, bucket_length, comm)

# generated longitudinal coordinate is z position (beta*c*dt) but Synergia uses
# c*dt.  Divide by beta to get c*dt.
local_particles = myBunch.get_local_particles()
local_particles[:,4] /= opts.beta

In [ ]:
pltbunch.plot_bunch(myBunch)
pltbunch.plot_long(myBunch)

## Run the simulation

In [ ]:
bunch_simulator = synergia.simulation.Bunch_simulator(myBunch)

#basic diagnostics - PER STEP
basicdiag = synergia.bunch.Diagnostics_basic("basic.h5", opts.output_dir)
bunch_simulator.add_per_step(basicdiag)

#include full diagnostics
fulldiag = synergia.bunch.Diagnostics_full2("full.h5", opts.output_dir)
bunch_simulator.add_per_turn(fulldiag)

#particle diagnostics - PER TURN
opts.turnsPerDiag = 1
particlediag = synergia.bunch.Diagnostics_particles("particles.h5",0,0,opts.output_dir)
bunch_simulator.add_per_turn(particlediag, opts.turnsPerDiag)

In [ ]:
opts.maxturns = opts.turns+1

propagator = synergia.simulation.Propagator(stepper)
propagator.set_checkpoint_period(opts.checkpointperiod)
propagator.propagate(bunch_simulator,opts.turns, opts.maxturns,opts.verbosity)

workflow.cleanup(opts.output_dir)

## Diagnostics

In [ ]:
opts.inputfile = opts.output_dir + '/basic.h5'
opts.plots = ['x_std', 'y_std']
plotVals = diagplot.getPlotVals(opts.inputfile, opts.plots)

#define specific value arrays
xmaster = plotVals['s']
xstd = plotVals['x_std']
ystd = plotVals['y_std']

fig = plt.figure(figsize=(8,6))
ax = plt.gca()
ax.plot(xmaster,xstd*1.e3,'b-', alpha=0.7, label = 'x_std') #plot x
ax.plot(xmaster,ystd*1.e3,'g-', alpha=0.7, label = 'y_std') #plot y
axtitle = "RMS envelope evolution over 50 m - 14.1 mA"
ax.set_title(axtitle, y = 1.02, fontsize = 18)  
ax.set_xlabel("s [m]",fontsize=14)
ax.set_ylabel("rms beam size $\sigma_x$ [mm]",fontsize=14)
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=14)
ax.legend()
sv_title = 'SC_test_envelope_10m_bunch.pdf'
fig.tight_layout()

In [ ]:
pltbunch.plot_bunch(myBunch)

In [ ]:
pltbunch.plot_long(myBunch)

In [ ]:
#Look at z distribution
part = myBunch.get_local_particles()
part[:,4]
zvals = part[:,4]
utils.plot_distribution(zvals, 100)

## Analytical Comparison

In [ ]:
def calc_perveance(I,ref,cn=0):
    '''Calculate the perveance for a proton beam of a given current and particle energy.
    
    Arguments
        - I - current in A
        - ref - the reference particle for extracting beta and gamma
        
        - (optional) charge neutralization factor - default 0
    '''
    
    I0 = 3.13e7 #characteristic current
    
    beta = ref.get_beta()
    gamma = ref.get_gamma()
    
    return (I/I0)*(2/beta**3)*(1/gamma**3)

def calc_characteristic_current():
    '''Return characteristics current for proton beam'''
    return 4*np.pi*scipy.constants.epsilon_0*scipy.constants.m_p*(scipy.constants.c**3)/scipy.constants.e

In [ ]:
#Introduce numerical integrators

#2nd Order RK - Ralston Method
def Ralston(r,z,h,f):
    k1 = h*f(r)
    return 0.25*k1 + 0.75*h*f(r+(2/3)*k1)

#4th Order Runge-Kutta
def RungeKutta4(r,z,h,f):
    k1 = f(r)
    k2 = f(r + (h/2)*k1)
    k3 = f(r + (h/2)*k2)
    k4 = f(r + h*k3)
    return h/6*(k1 + 2*k2 +2*k3 + k4)

#function here, which is a function of r and z
def rprime(K,emit,r0,rp0,rm):
    '''
    
    Returns the slope of the beam envelope (dr/dz) for a given value of emittance,rm, K, and initial conditions.
    
    This equation follows from Reisier.
    
    Arguments:
    
        - r - beam radius (or RMS)
        - K - perveance
        - emit - geometric emittance
        - r0 - initial envelope radius (or RMS)
        - rp0 - initial slope of envelope (or RMS)
        
    '''
    
    first = rp0**2 #first term
    second = (emit**2)*((1./r0**2)-(1./rm**2)) #second term
    third = 2*K* np.log(rm/r0) / 4
    
    return np.sqrt(first + second + third)

In [ ]:
import math
from __future__ import division

def calculate_expansion(current, reference_paricle,r0,rp0,emit=emit,N=1000,zf=opts.turns * lattice.get_length()):

    '''Evaluate the expansion of a KV beam envelope in a drift along z-axis, begining at z = 0.
    
    Arguments:
        - current - beam current in A
        - reference_particle - synergia object for bunch/lattice reference particle
        - r0 - initial envelope value (provide RMS for RMS expansion, a for envelope expansion, etc.)
        - rp0 - initial slope of envelope (must be non-zero, but calculation is not sensitive to small values)
        
        - (optional) emit - geometric emittance of beam - default 2.05721258396*1.e-6 (for 0.3 mm-mrad KV beam)
        - (optional) N - number of steps for integration - default 1000
        - (optional) zf - final z value (e.g. length of expansion) - default 50.0
        
    '''
    
    z0 = 0.0 #start
    ss = (zf-z0)/N #step size

    zpoints = np.linspace(0.0, zf, num=N) #define z values
    rpoints = [] #empty array for r values
    
    #calculate perveance
    Kp = calc_perveance(current, reference_particle)
    
    #x is r
    #z is t (what we step up)
    #f is our function describing the relationship between r and z
    f = lambda r: rprime(Kp,emit,r0,rprime0,r)

    r,z,dz = r0,z0,ss
    points = []
    while z < zf:
        points.append((z,r))
        z, r = z+dz, r + Ralston(r,z,dz,f) #incremement
        
    return points

In [ ]:
#Calculate current - 14 mA 

current14 = 14*1.e-3
rprime0 = 1.0*(xstd[1]-xstd[0])/(xmaster[1]-xmaster[0])
r0 = xstd[0] #1.0*1.e-3 #initial envelope value
#emit = 4.10849449506e-06 #not used (hard coded into calculate_expansion) #gemit_x  #rms geometric emittance
points14 = calculate_expansion(current14, reference_particle, r0,rprime0)
points0 = calculate_expansion(0.0, reference_particle, r0,rprime0)
print emit

In [ ]:
#Compare the results
fig = plt.figure(figsize=(8,6))
ax = plt.gca()
ax.plot(xmaster,xstd*1.e3,'b-', alpha=0.7, label = 'simulation - 14mA') #plot x
ax.plot([p[0] for p in points14], [p[1]*1.e3 for p in points14],'g--',alpha=0.7, label = 'theory - 14 mA')
ax.plot([p[0] for p in points0], [p[1]*1.e3 for p in points0],'k--',alpha=0.7, label = 'theory - zero current')
axtitle = "RMS envelope over %s m - theory vs simulation" % (opts.turns * lattice.get_length())
ax.set_title(axtitle, y = 1.02, fontsize = 18)  
ax.set_xlabel("s [m]",fontsize=14)
ax.set_ylabel("rms beam size $\sigma_x$ [mm]",fontsize=14)
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=14)
ax.legend(loc = 2)
fig.tight_layout()